In [ ]:
%%time
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'start.ipynb'
from baseline import train_bert, check_bert
import wandb
wandb.login(key="68be83ff5b81233c60ffad40f6327a3700797a16")
#check_bert()
train_bert()

In [ ]:
import pandas as pd
import polars as pl
import numpy as np
from tqdm import tqdm
from related_split import related_split_train_valid

# Для примера создадим DataFrame
data = {
    'variantid1': np.random.randint(1, 100, 30),
    'variantid2': np.random.randint(1, 100, 30)
}

train_data = pd.DataFrame(data)


In [ ]:
train_split, valid_split = related_split_train_valid(train_data)
train_split

In [ ]:
valid_split

In [ ]:
train_data

In [ ]:
import joblib
import pandas as pd
train_idx = joblib.load('./data/train_idx.pkl')
val_idx = joblib.load('./data/val_idx.pkl')
train_data_all = joblib.load('./data/train_data_all.pkl')

In [ ]:
print(train_data_all.shape)
print(train_data.shape)
print(val_data.shape)

In [ ]:
import joblib
val_data = joblib.load('./data/val_data.pkl')
train_data = joblib.load('./data/train_data.pkl')

In [ ]:
val_data.head()

In [ ]:
val_data.columns

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Предположим, что val_data содержит ваши эмбеддинги
# Получаем эмбеддинги
embedding_1 = val_data.loc[2]['name_bert_1']
embedding_2 = val_data.loc[2]['name_bert_2']

# Преобразуем эмбеддинги в двумерные массивы (требуется для cosine_similarity)
embedding_1 = embedding_1.reshape(1, -1)
embedding_2 = embedding_2.reshape(1, -1)

# Вычисляем косинусное сходство
similarity = cosine_similarity(embedding_1, embedding_2)

print(f"Косинусное сходство: {similarity[0, 0]}")

In [ ]:
type(val_data.loc[0]['name_bert_1'])

In [ ]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(train_data_all, test_size=0.2, random_state=42)

In [ ]:
# Проверка пересекающихся индексов
intersecting_indices = val_data.index.intersection(train_data.index)
if not intersecting_indices.empty:
    print("Есть пересекающиеся индексы между val_data и train_data:", intersecting_indices)
else:
    print("Нет пересекающихся индексов между val_data и train_data.")

# Проверка одинаковых идентификаторов в столбцах variantid1
intersecting_variantid1 = set(val_data['variantid1']).intersection(set(train_data['variantid1']))
intersecting_variantid12 = set(val_data['variantid1']).intersection(set(train_data['variantid2']))
if intersecting_variantid1 or intersecting_variantid12:
    print("Есть пересекающиеся идентификаторы в столбце variantid1:", intersecting_variantid1)
else:
    print("Нет пересекающихся идентификаторов в столбце variantid1.")

# Проверка одинаковых идентификаторов в столбцах variantid2
intersecting_variantid2 = set(val_data['variantid2']).intersection(set(train_data['variantid2']))
intersecting_variantid21 = set(val_data['variantid2']).intersection(set(train_data['variantid1']))
if intersecting_variantid2 or intersecting_variantid21:
    print("Есть пересекающиеся идентификаторы в столбце variantid2:", intersecting_variantid2)
else:
    print("Нет пересекающихся идентификаторов в столбце variantid2.")

In [ ]:
val_data.head()

In [ ]:
train_data.head()

In [ ]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine

# Два текста для сравнения
#text1 = "Пример первого текста для анализа сходства."
#text2 = "Этот текст является вторым примером для определения похожести."
text1 = "112"
text2 = "223"



# Создание экземпляра TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Преобразование текстов в TF-IDF векторы
tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])

# Вычисление косинусного сходства между двумя текстовыми векторами
similarity_score = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])

tfidf_matrix = tfidf_vectorizer.fit_transform([text3, text4])


from scipy.spatial.distance import cosine

print(f"Cosine Similarity between texts: {similarity_score[0][0]}")


In [ ]:
tfidf_matrix[1:2]

In [ ]:
val_data = joblib.load('./data/val_data.pkl')

In [ ]:
val_data.head(1)

In [ ]:
val_data.loc[4]['main_pic_embeddings_resnet_v1']

In [ ]:
val_data.loc[4]['pic_embeddings_resnet_v1']

In [ ]:
val_data.loc[3]['pic_embeddings_1']


In [ ]:
avg_embedding1 = np.mean(val_data.loc[3]['pic_embeddings_1'], axis=0)

In [ ]:
avg_embedding1

In [ ]:
import numpy as np

# Функция для объединения всех вложенных numpy.ndarray в один массив с сохранением структуры
def combine_embeddings(row):
    # Извлечение массивов из колонок
    embed1 = row['main_pic_embeddings_resnet_v1']
    embed2 = row['pic_embeddings_resnet_v1']
    
    # Создаем пустой список для хранения всех вложенных массивов
    combined_embeds = []
    
    # Если поле не пустое, добавляем его вложенные массивы в общий список
    if isinstance(embed1, np.ndarray):
        combined_embeds.extend(embed1)  # Добавляем все элементы из embed1
    if isinstance(embed2, np.ndarray):
        combined_embeds.extend(embed2)  # Добавляем все элементы из embed2
    
    # Преобразуем список обратно в numpy массив
    return np.array(combined_embeds)

# Применение функции к DataFrame и обновление колонки main_pic_embeddings_resnet_v1
val_data['main_pic_all'] = val_data.apply(combine_embeddings, axis=1)

In [ ]:
import joblib
import pandas as pd
#text_and_bert = pd.read_parquet('./data/train/text_and_bert.parquet')
attributes = pd.read_parquet('./data/train/attributes.parquet')

In [ ]:
attributes.head()

In [ ]:
attributes.loc[0]['categories']

In [ ]:
attributes.loc[0]['categories']

In [ ]:
import json
from tqdm import tqdm

# Создание словарей для хранения уникальных значений по каждой категории
unique_categories = {"1": set(), "2": set(), "3": set(), "4": set()}

# Проход по всем строкам DataFrame
for index, row in attributes.iterrows():
    categories_json = row['categories']
    categories_dict = json.loads(categories_json)
    
    # Добавление уникальных значений для каждой категории в соответствующее множество
    for key in unique_categories.keys():
        category_name = categories_dict.get(key)
        if category_name:  # Если значение существует, добавляем в множество
            unique_categories[key].add(category_name)

# Печать количества уникальных значений и сами значения для каждой категории
for key, values in unique_categories.items():
    print(f"Количество уникальных названий в категории '{key}': {len(values)}")
    print(f"Все уникальные названия в категории '{key}': {values}")
    print()

In [ ]:
%%time
for i in range(10000):
    #get_cosine_similarity(embed1,embed2)
    d= 1- cosine(embed1,embed2)

In [ ]:
CPU times: total: 2.5 s
Wall time: 3.79 s

In [ ]:
import pandas as pd

# Пример функции, которая принимает строку и дополнительные аргументы
def make_features_for_row(row, arg1, arg2):
    # Ваша логика создания признаков
    feature = row['column1'] * arg1 + row['column2'] * arg2
    return feature

# Создание примерного DataFrame
data = {'column1': [1, 2, 3], 'column2': [4, 5, 6]}
df = pd.DataFrame(data)

# Применение функции с передачей дополнительных аргументов
df['features'] = df.apply(make_features_for_row, axis=1, args=(10, 20))

print(df)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
#tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
#model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased')
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    #embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

print('***',embed_bert_cls('привет мир', model, tokenizer).shape)

In [ ]:
%%time
from transformers import BertModel, BertTokenizer
import torch
from scipy.spatial.distance import cosine

# Загрузка модели и токенизатора BERT
model_name = 'DeepPavlov/rubert-base-cased'
#model_name = 'cointegrated/rubert-tiny'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)  # Используйте BertModel, а не BertForSequenceClassification
#model = torch.load ('bert.pkl')

# Функция для создания эмбеддингов BERT
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Используйте last_hidden_state для извлечения эмбеддингов
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Пример использования
# for i in range(300):
#     text = "Пример 1"
#     embedding = get_bert_embedding(text)
#print(embedding)
embed1 = get_bert_embedding('Полотенце детское')
#embed2 = get_bert_embedding('Полотенцесушитель')
embed2 = get_bert_embedding('Парфюмерная вода')
dist = 1-cosine(embed1, embed2)
print(dist)

In [ ]:
%%time
from transformers import BertModel, BertTokenizer
import torch
from scipy.spatial.distance import cosine

# Загрузка модели и токенизатора BERT
#model_name = 'DeepPavlov/rubert-base-cased'
#model_name = 'cointegrated/rubert-tiny'
model_name = 'cointegrated/rubert-tiny2'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)  # Используйте BertModel, а не BertForSequenceClassification

# Функция для создания эмбеддингов BERT
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Используйте last_hidden_state для извлечения эмбеддингов
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Пример использования
# for i in range(300):
#     text = "Пример 1"
#     embedding = get_bert_embedding(text)
embedding = get_bert_embedding('Набор для рукоделия, творчества')
# print(embedding)
#embed1 = get_bert_embedding('Полотенце детское')
embed1 = get_bert_embedding('100')
#embed2 = get_bert_embedding('Полотенцесушитель')
#embed2 = get_bert_embedding('Парфюмерная вода')

dist = 1-cosine(embed1, embed2)
print(dist)

In [ ]:
import joblib
val_data = joblib.load('./data/val_data.pkl')

In [ ]:
val_data.shape
val_data_head = val_data.head(10)

In [ ]:
val_data_head.shape

In [ ]:
val_data_head.head

In [ ]:
%%time
import json
from transformers import BertModel, BertTokenizer
import torch

# Загрузка модели и токенизатора BERT
# model_name = 'bert-base-uncased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = BertModel.from_pretrained(model_name)

model_name = 'cointegrated/rubert-tiny2'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name) 

# Функция для создания эмбеддингов BERT
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# Создание множества для хранения всех уникальных значений категорий
unique_values = set()

# Проход по всем строкам DataFrame и сбор уникальных значений
for index, row in val_data_head.iterrows():
    # Добавляем все значения категорий в множество уникальных значений
    unique_values.update(row['categories_1'].values(),
                        row['categories_2'].values())

# Создание словаря эмбеддингов категорий, где ключи - уникальные значения, а значения - их эмбеддинги BERT
categories_embeddings_dict = {value: get_bert_embedding(value) for value in unique_values}

# Пример вывода
for key, embedding in embeddings_dict.items():
    print(f"Ключ: {key}")
    print(f"Эмбеддинг: {embedding[:5]}...")  # Выводим первые 5 значений эмбеддинга для краткости
    print()

In [ ]:
i = 25
print(val_data.loc[i]['categories_1'])
print(val_data.loc[i]['categories_2'])

In [ ]:
val_data_head.loc[0]['categories_1']['1']

In [ ]:
val_data_head.loc[0]['categories_1']['1']

In [ ]:
category_distance

In [ ]:
category_distance['2']

In [ ]:
val_data_head.loc[0]['features']

In [ ]:
val_data.loc[25]['features']

In [ ]:
i = 25
print(val_data.loc[0:10]['characteristic_attributes_mapping_1'])

In [ ]:
import pandas as pd
from collections import defaultdict

# Предположим, val_data уже загружен как DataFrame

# Создание словаря для хранения частоты встречаемости ключей
key_frequency = defaultdict(int)

# Проход по всем строкам в val_data и подсчёт ключей
for i, row in val_data.iterrows():
    # Учитываем ключи из первой колонки
    if pd.notnull(row['characteristic_attributes_mapping_1']):
        for key in row['characteristic_attributes_mapping_1'].keys():
            key_frequency[key] += 1
    
    # Учитываем ключи из второй колонки
    if pd.notnull(row['characteristic_attributes_mapping_2']):
        for key in row['characteristic_attributes_mapping_2'].keys():
            key_frequency[key] += 1

# Сортировка ключей по частоте встречаемости в порядке убывания
sorted_key_frequency = sorted(key_frequency.items(), key=lambda item: item[1], reverse=True)

# Вывод результатов
print(f"Количество уникальных ключей: {len(sorted_key_frequency)}")

# Вывод результатов
print("Ключи отсортированы по убыванию встречаемости:")
# for key, freq in sorted_key_frequency:
#     print(f"{key}: {freq} раз(а)")


In [ ]:
val_data.shape

In [ ]:
val_data.columns

In [ ]:
val_data[['characteristic_attributes_mapping_1','characteristic_attributes_mapping_2']]

In [ ]:
import joblib
val_data = joblib.load('./data/result_data.pkl')

In [ ]:
import pandas as pd

# Предположим, val_data уже загружен как DataFrame

# Извлечение нужных колонок
# columns_to_export = val_data[['characteristic_attributes_mapping_1', 'characteristic_attributes_mapping_2', 'name_1', 'name_2', 'target', 'y_pred', 'y_pred_prob', 'features']]

# Запись в текстовый файл
with open('output.txt', 'w', encoding='utf-8') as file:
    for index, row in val_data.iterrows():
        file.write(f"Row: {index}:, Target: {row['target']}, y_pred: {row['y_pred']}, y_pred_prob: {row['y_pred_prob']}:")
        if row['target'] == row['y_pred']:
            file.write(f"  Good\n")
        else:
            file.write(f"  Mistake!!!\n")
        file.write(f"Name_1: {row['name_1']}:\n")
        # Запись ключей и значений из characteristic_attributes_mapping_1
        if pd.notnull(row['characteristic_attributes_mapping_1']):
            file.write("characteristic_attributes_mapping_1:\n")
            for key, value in row['characteristic_attributes_mapping_1'].items():
                file.write(f"  * {key}: {', '.join(value)}\n")
        
        file.write(f"+++++:\n")
        file.write(f"Name_2: {row['name_2']}:\n")
        # Запись ключей и значений из characteristic_attributes_mapping_2
        if pd.notnull(row['characteristic_attributes_mapping_2']):
            file.write("characteristic_attributes_mapping_2:\n")
            for key, value in row['characteristic_attributes_mapping_2'].items():
                file.write(f"  * {key}: {', '.join(value)}\n")

        feature_names = [
            
        ]
        for i, feature in enumerate(row['features']):
        
        file.write("***********************************************\n")  # Добавляем пустую строку между записями


In [ ]:
with open('pattern_attributes.txt', 'r', encoding='utf-8') as file:
    exact_attributes = [line.strip() for line in file]
print('Начальная длина списка атрибутов:', len(exact_attributes))

In [ ]:
val_data.loc[13]['categories_1']

In [ ]:
val_data.loc[13]['categories_2']

In [ ]:
import joblib
text_and_bert = pd.read_parquet('./data/train/text_and_bert.parquet')

In [ ]:
text_and_bert.columns

In [ ]:
val_data.

In [ ]:
# Возвращает атрибуты в виде словаря переделанные в строку строку 
def get_characteristic_attributes_mapping_string(characteristic_attributes_mapping):
    ret_str = ''
    for key, value in characteristic_attributes_mapping.items():
        ret_str += f"{key}: {', '.join(value)} "
    return ret_str

ret1 = get_characteristic_attributes_mapping_string(val_data.loc[3]['characteristic_attributes_mapping_1'])
ret2 = get_characteristic_attributes_mapping_string(val_data.loc[3]['characteristic_attributes_mapping_2'])

In [ ]:
len(ret1)

In [ ]:
int(ret1 != ret2)

In [ ]:
val_data.loc[0]['characteristic_attributes_mapping_1'] == val_data.loc[0]['characteristic_attributes_mapping_2']

In [ ]:
len(val_data.loc[13]['characteristic_attributes_mapping_1']['Артикул'][0])

In [ ]:
len(val_data.loc[1]['characteristic_attributes_mapping_1'])

In [ ]:
ret = []
ret.extend([1,2])

In [ ]:
ret

In [ ]:
dict1 = {'a': 1, 'b': 2, 'c': 3}
dict2 = {'a': 1, 'b': 20, 'd': 4}

# Полное сравнение
print(dict1 == dict2)  # False

# Сравнение ключей
print(dict1.keys() == dict2.keys())  # False

# Вывод различий
only_in_dict1 = dict1.keys() - dict2.keys()  # {'c'}
only_in_dict2 = dict2.keys() - dict1.keys()  # {'d'}
differing_values = {key: (dict1[key], dict2[key]) for key in dict1.keys() & dict2.keys() if dict1[key] != dict2[key]}  # {'b': (2, 20)}

print(f"Ключи только в dict1: {only_in_dict1}")
print(f"Ключи только в dict2: {only_in_dict2}")
print(f"Различия в значениях: {differing_values}")

In [ ]:
float(int(1 == 1))

In [ ]:
import Levenshtein

str1 = 'H47436-2|R06Z'
str2 = 'H47436-4|R04Z'

# Вычисление расстояния Левенштейна
distance = Levenshtein.distance(str1, str2)
print(f"Расстояние Левенштейна: {distance}")

In [ ]:
my_dict = {'a': 'asdfasdf', 'b': 2, 'c': 3}
# Удаление ключа 'b'
del my_dict['b']

print(f"Обновлённый словарь: {my_dict}")

In [ ]:
len(my_dict['a'])

In [ ]:
import re

# Слово, которое нужно найти
word_to_find = 'слово'

# Пример строки
text = "Это тестовое слово в середине слово строки, и ещё одно слово в конце."

# Регулярное выражение для поиска слова
pattern = rf'\b{word_to_find}\b'

# Поиск совпадений
matches = re.findall(pattern, text)

# Вывод результатов
print(f"Найденные совпадения: {matches}")


In [ ]:
if matching_keys:
    print('1')

In [ ]:
# Пример двух словарей
attr_1 = {'a': 1, 'b': 2, 'c': 3}
attr_2 = {'b': 4, 'c': 5, 'd': 6}

# Находим общие ключи
attr_1_and_2 = attr_1.keys() & attr_2.keys()
print("Общие ключи:", attr_1_and_2)

# Шаблон для поиска и удаления ключа
key_to_remove = 'b'

# Проверяем, есть ли ключ в общих ключах, и удаляем его, если есть
if key_to_remove in attr_1_and_2:
    print(f"Ключ '{key_to_remove}' найден в общих ключах и будет удален")
    attr_1_and_2.remove(key_to_remove)

print("Оставшиеся общие ключи:", attr_1_and_2)

In [ ]:
import re

# Пример списков
exact_attributes = ['attr1', 'attr2', 'attr_special', 'some_other_attr', 'another_special_attr']
pattern_attributes = [r'.*_special$', r'^attr\d$']

# Фильтрация списка exact_attributes
filtered_attributes = [
    attr for attr in exact_attributes
    if not any(re.match(pattern, attr) for pattern in pattern_attributes)
]

print("Отфильтрованный список:", filtered_attributes)

In [ ]:
with open('exact_attributes.txt', 'r', encoding='utf-8') as file:
        exact_attributes = [line.strip() for line in file]

In [ ]:
len(exact_attributes)

In [ ]:
exact_attributes[499]

In [ ]:
%%time
from baseline import main

In [ ]:
import joblib
val_features = joblib.load('./data/val_features.pkl.gz')

In [ ]:
val_features.shape

In [ ]:
import numpy as np
from scipy import sparse

# Пример плотной матрицы NumPy
# Преобразование плотной матрицы в разреженную матрицу формата CSR (Compressed Sparse Row)
sparse_matrix = sparse.csr_matrix(val_features)

# Проверяем размер занимаемой памяти
print(f"Размер плотной матрицы: {val_features.nbytes} байт")
print(f"Размер разреженной матрицы: {sparse_matrix.data.nbytes + sparse_matrix.indptr.nbytes + sparse_matrix.indices.nbytes} байт")

# Если нужно преобразовать обратно в плотную матрицу
dense_matrix_back = sparse_matrix.toarray()

In [ ]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(sparse_matrix, test_size=0.2, random_state=42)

In [ ]:
type(train_data)

In [ ]:
val_data.nbytes

In [ ]:
import catboost
print(catboost.__version__)

In [ ]:
from scipy import sparse
from sklearn.model_selection import train_test_split

# Предположим, что sparse_matrix уже создана и является разреженной матрицей формата CSR
sparse_matrix = sparse.csr_matrix(val_features)

# Разбиваем разреженную матрицу на тренировочную и тестовую выборки
train_data, val_data = train_test_split(sparse_matrix, test_size=0.2, random_state=42, shuffle=True)

# Обратите внимание, что если вы работаете с разреженными матрицами, `train_test_split` вернет их в разреженном формате
print(f"Тип train_data: {type(train_data)}")
print(f"Тип val_data: {type(val_data)}")

In [ ]:
X_train = joblib.load('./data/X_train.pkl.gz')

In [ ]:
X_train.shape

In [ ]:
train_data = joblib.load('./data/train_data.pkl.gz')

In [ ]:
train_data.shape

In [ ]:
import pandas as pd
train_path = './data/train/train.parquet'
train = pd.read_parquet(train_path)

In [ ]:
train.shape

In [ ]:
X_train[0]

In [ ]:
dense_array = X_train[0].toarray()
print(dense_array)

In [ ]:
!nvidia-smi

In [ ]:
# Преобразуем первую строку в плотный массив (dense array)
dense_array = X_train[0].toarray()

# Получаем индексы ненулевых элементов
nonzero_indices = dense_array.nonzero()

# Печатаем ненулевые элементы и их индексы
for index in zip(nonzero_indices[1], dense_array[0, nonzero_indices[1]]):
    print(f"Index: {index[0]}, Value: {index[1]}")

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Пример генерации разреженных данных
from scipy.sparse import csr_matrix
import numpy as np

# Генерация случайных разреженных данных
n_samples = 1000000
n_features = 10000
X_sparse = csr_matrix(np.random.binomial(1, 0.01, size=(n_samples, n_features)))
y = np.random.randint(0, 2, size=n_samples)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_sparse, y, test_size=0.2, random_state=42)

# Настройка гиперпараметров LightGBM
params = {
    'boosting_type': 'goss',  # Gradient-based One-Side Sampling, работает лучше на разреженных данных
    'objective': 'binary',  # Двухклассовая классификация
    'metric': 'binary_logloss',  # Логарифмическая потеря
    'num_leaves': 64,  # Количество листьев в каждом дереве
    'max_depth': -1,  # Без ограничения глубины
    'learning_rate': 0.05,  # Скорость обучения
    'feature_fraction': 0.7,  # Доля признаков, используемых при построении каждого дерева
    'bagging_fraction': 0.8,  # Доля данных для бэггинга
    'bagging_freq': 5,  # Частота бэггинга
    'min_data_in_leaf': 50,  # Минимальное количество данных в листе
    'lambda_l1': 0.1,  # L1-регуляризация
    'lambda_l2': 0.1,  # L2-регуляризация
    'max_bin': 255,  # Максимальное количество бинов для разбиения
    'verbose': -1  # Отключить вывод информации о процессе обучения
}

# Подготовка данных для LightGBM
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Обучение модели
model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[test_data], early_stopping_rounds=50)

# Прогнозирование и оценка точности
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]
accuracy = accuracy_score(y_test, y_pred_binary)

print(f'Accuracy: {accuracy:.4f}')

In [ ]:
dict1 = {'Диаметр, мм': (['0.28'], ['0.28'])}

In [ ]:
dict1 = {'Диаметр, мм': ['0.28', '0.29']}

In [ ]:
type(dict1['Диаметр, мм'])

In [ ]:
if isinstance(dict1['Диаметр, мм'], list):
    out = ' '.join(map(str, dict1['Диаметр, мм']))
else:
    out = str(dict1['Диаметр, мм'])

In [ ]:
out

In [ ]:
attributes_text = ' '.join(
            [' '.join(map(str, v)) if isinstance(v, list) else str(v) for v in dict1['Диаметр, мм']]
        )

In [ ]:
type(attributes_text)

In [ ]:
attributes_text

In [ ]:
map(str, dict1['Диаметр, мм'])

In [ ]:
' '.join(map(str, ['0.28', '0.28']))

In [ ]:
import re
def get_texts_similarity(text1, text2):
        '''
        Возвращает сходство между текстами text1 и text2
        '''
        # Создание экземпляра TfidfVectorizer
        tfidf_vectorizer = TfidfVectorizer(
            token_pattern=r'\b\w+\b',  # Учитывать любые слова, включая цифры
            stop_words=None  # Не использовать встроенные стоп-слова
        )
        # Преобразование текстов в TF-IDF векторы
        # print('text1:', text1, 'text2:', text2)
        match1 = re.search(r'^\d+$', text1)
        match2 = re.search(r'^\d+$', text2)
        if match1 and match2:
            # Если это числа возвращаем соотношение
            match1 = int(match1.group())
            match2 = int(match2.group())
            return(min(match1,match2)/max(match1,match2))
        else:
            try:
                tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])
                return cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
            except ValueError as e:
                return 0

In [ ]:
get_texts_similarity('100', '99')

In [ ]:
match1 = 0
match2 = 0
if max(match1,match2) != 0:
    a = (min(match1,match2)/max(match1,match2))
else:
    a =0
a

In [ ]:
import joblib
train_data = joblib.load('./data/val_data.pkl')

In [ ]:
from tqdm import tqdm
categories_list = [[] for _ in range(4)]
for i, (index, row) in enumerate(tqdm(train_data.iterrows(), total=len(train_data), desc="Processing rows", ncols=100, leave=True)):
    for j in range(1,5):
        categories_list[j-1].append(str(row[f'categories_1'][f'{j}']))
        categories_list[j-1].append(str(row[f'categories_2'][f'{j}']))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
#for i in range(4):
for i in range(2):
    print(f"*************** Заголовок {i} ******************")
    # Слова и их индексы
    tfidf_vectorizer = TfidfVectorizer(max_features=6000)
    tfidf_vectorizer.fit(categories_list[i])
    print("Длина словаря:", len(tfidf_vectorizer.vocabulary_))
    print("Словарь (vocabulary):")
    print(tfidf_vectorizer.vocabulary_)
    # IDF значения для каждого слова
    print("\nIDF значения для слов:")
    print(dict(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_)))


In [ ]:
#!
from scipy.sparse import lil_matrix, hstack, csr_matrix
at = csr_matrix(tfidf_vectorizer.transform(['спорт товары']).toarray())
at[at > 0] = 1
at[0]

In [ ]:
print(at.toarray())

In [ ]:
train_data.loc[0]['categories_1']['1']

In [ ]:
train_data.loc[0]['categories_1']['1']

In [ ]:
categories_list[3]

In [ ]:
from baseline import make_exact_attributes_from_train
make_exact_attributes_from_train()

In [ ]:
from baseline import load_features
X_train, X_val, y_train, y_val = load_features()

In [ ]:
type(y_train)

In [ ]:
X_val

In [ ]:
import lightgbm as lgb
import optuna
from sklearn.metrics import precision_recall_curve, auc
from lightgbm import LGBMClassifier
from optuna.integration import LightGBMPruningCallback
import numpy as np

def objective(trial):
    # Определяем параметры модели
    params = {
        'objective': 'binary',
        #'metric': 'binary_logloss',
        'metric': 'auc',
        'num_leaves': trial.suggest_int('num_leaves', 31, 255),
        'max_depth': trial.suggest_int('max_depth', -1, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10)
    }


    # Обучаем модель
    model = LGBMClassifier(**params)
    # model.fit(X_train, y_train, 
    #           eval_set=[(X_val, y_val)],
    #           early_stopping_rounds=50, 
    #           verbose=False,
    #           callbacks=[LightGBMPruningCallback(trial, 'binary_logloss')])

    # model.fit(X_train, y_train, verbose=False)
    # model.fit(X_train, y_train, 
    #           callbacks=[lgb.early_stopping(stopping_rounds=50),])
    
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
    model = lgb.train(params,
                      train_data,
                      num_boost_round=1000,
                      valid_sets=[val_data],
                      callbacks=[lgb.early_stopping(stopping_rounds=50),]
                     )
    # Получаем прогнозы
    y_pred_prob = model.predict(X_val)
    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
    prauc = auc(recall, precision)

    return prauc

In [ ]:
%%time
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

# Выводим лучшие параметры
print("Best parameters:", study.best_params)
print("Best score:", study.best_value)

In [ ]:
%%time
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=800)

# Выводим лучшие параметры
print("Best parameters:", study.best_params)
print("Best score:", study.best_value)

In [ ]:
import joblib
val_features = joblib.load('./data/val_features.pkl')

In [ ]:
val_features.shape

In [ ]:
result_data = joblib.load('./data/result_data.pkl')
result_data.reset_index(drop=True, inplace=True)

In [ ]:
result_data.shape

In [ ]:
val_features1 = val_features.toarray()

In [ ]:
val_features1.shape

In [ ]:
def round_to_nearest(value, targets=[0.1, 0.2, 0.6, 0.8, 0.9, 0.95, 0.97, 1]):
    return min(targets, key=lambda x: abs(x - value))

# Пример использования
number = 0.99
rounded_number = round_to_nearest(number)
print(rounded_number)

In [ ]:
import re

def extract_words(text):
    # Ищем слова длиной 5 и более символов, содержащие не менее трех цифр
    pattern = r'\b\w{5,}\b'  # ищем слова длиной 5 и более символов
    words = re.findall(pattern, text)
    
    # Фильтруем слова, чтобы оставить только те, в которых не менее трех цифр
    result = [word for word in words if len(re.findall(r'\d', word)) >= 3]
    
    return result

# Пример использования
text = "There are some words like 123abc, xyz789, and 12a3b4c which should be found."
filtered_words = extract_words(text)
print(filtered_words)

In [ ]:
def find_common_words(list1, list2):
    # Преобразуем списки в множества и находим пересечение
    common_words = set(list1) & set(list2)
    return list(common_words)

# Пример использования
list1 = ["apple123", "banana456", "grape7891"]
list2 = ["banana456", "orange123", "grape789", "melon234"]

common_words = find_common_words(list1, list2)
print(common_words)

In [ ]:
if len(common_words) > 0:
    print('1')

In [ ]:
def extract_words(text):
    # Ищем слова длиной 5 и более символов, содержащие не менее трех цифр
    # pattern = r'\b\w{5,}\b'  # ищем слова длиной 5 и более символов
    # 1. \b\w{3,}\b - слова длиной не менее 3 символов с цифрами или английскими буквами
    # 2. \b\d{7,}\b - слова, состоящие только из цифр, длиной не менее 7 символо
    #pattern = r'\b(?:(?:[A-Z]\d|\d[A-Z])|\d{7,})\b'

    # Исходный текст    
    # Удаляем слова лишние
    text = re.sub(r'\модель\b|\bеще что удалить\b', '', text)
    
    #pattern = r'\b(?:(?:[A-ZА-Я\d\s\-]*?(?:[A-ZА-Я]\d|[A-ZА-Я]{3,})|(?:\d[A-ZА-Я]|[A-ZА-Я]{3,})[А-ЯA-Z\d\s\-]*?)|\d{7,})\b'
    #pattern = r'\b(?:(?:[A-ZА-Я\d]*?[A-ZА-Я]\d|\d[A-ZА-Я][A-ZА-Я\d]*?)|\d{7,})\b'
    pattern = r'\b([А-ЯA-Z\d\s\-\/]+)\b'
    #words = re.findall(pattern, text)
    words = re.findall(pattern, text)
    
    # Фильтруем слова, чтобы оставить только те, в которых не менее трех цифр
    #result = [re.sub(r'[\s+\-]', ' ', word).strip() for word in words if len(word) >= 7]
    result = [re.sub(r'[\s\-]+', ' ', word).strip() for word in words if len(word.strip()) >= 7]
    
    return result

def find_common_words(list1, list2):
    # Преобразуем списки в множества и находим пересечение
    common_words = set(list1) & set(list2)
    return list(common_words)

In [ ]:
# name_1 = '2шт роликовая щетка для пылесоса Pro FC6 XC8 XC7:'
# name_2 = '2 шт. роликовая щетка, совместимая с пылесосом Pro FC6 XC8 XC7'
name_1 = 'Сплит-система AUX ASW-H12B4/FJ-SR1 AS-H12B4/FJ-R1 Серия FJ предназначена для помещения площадью 35-40 кв.метров. Этот кондиционер способен охладить/обогреть небольшое офисное помещение или квартиру из одной/двух комнат, общей площадью не более 35-40 м2. Прекрасно подойдет для просторной гостиной, а также небольшого коммерческого помещения.***Технические характеристики и внешний вид оборудования, приведенного в данном каталоге, могут быть изменены производителем без предварительного уведомления.:'
name_2 = 'Сплит-система AUX ASW-H18B4/FJ-SR1 AS-H18B4/FJ-R1 Серия FJ предназначена для помещения площадью до 50 кв.метров. Сплит-системы этой мощности нашли широкое применение в частных домах, многокомнатных квартирах, коммерческих объектах. Не рекомендуется устанавливать в спальных комнатах. Это хороший и производительный кондиционер, который способен как охлаждать, так и обогревать достаточно объемные помещения.***Технические характеристики и внешний вид оборудования, приведенного в данном каталоге, могут быть изменены производителем без предварительного уведомления.:'
list_1 = extract_words(name_1)
list_2 = extract_words(name_2)

In [ ]:
list_1

In [ ]:
list_2

In [ ]:
if re.search(r'указан.{,7}упаковк', 'указан на упаковке товара'):
    print(1)

In [ ]:
pattern = r'указан.{,7}упаковк'
text_1 = 'Состав: полный состав указан на упаковке товара'
text_2 = 'Состав: полный состав указан на упаковке товара'
if ((re.search(pattern, text_1) and (len(text_1) < 100)) or
    (re.search(pattern, text_2) and (len(text_2) < 100))):
    print(1)

In [ ]:
if ((re.search(pattern, text_1)) or
    (re.search(pattern, text_2))):
    print(1)

In [ ]:
len(text_1)

In [ ]:
# Исходная строка на русском
my_string = "пример"

# Получение длины строки в символах
length = len(my_string)

print(length)

In [ ]:
from baseline import load_features
#X_train,  y_train, X_val, y_val = load_features()
X_train, X_val, y_train, y_val = load_features()

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
from baseline import adjust_class_balance
X_train, y_train = adjust_class_balance(X_train, y_train, remove_fraction=0.2)

In [ ]:
from baseline import adjust_class_balance
X_balanced, y_balanced = adjust_class_balance(X_val, y_val, remove_fraction=0.75)

In [ ]:
from collections import Counter

# Предположим, что y_val - это массив меток классов
class_balance = Counter(y_val)
print("Баланс классов:", class_balance)

In [ ]:
# Предположим, что y_val - это массив меток классов
class_balance = Counter(y_balanced)
print("Баланс классов:", class_balance)

In [ ]:
# Предположим, что y_val - это массив меток классов
class_balance = Counter(y_balanced)
print("Баланс классов:", class_balance)

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

def adjust_class_balance(X, y, remove_fraction):
    """
    Удаляет определенную долю строк с меткой 1, чтобы изменить баланс классов.
    
    :param X: разряженная матрица признаков (csr_matrix)
    :param y: метки классов (numpy array)
    :param remove_fraction: доля строк с меткой 1, которую нужно удалить (значение от 0 до 1)
    :return: измененные X и y
    """
    # Находим индексы строк с меткой 1
    indices_one = np.where(y == 0)[0]
    
    # Определяем количество строк, которые нужно удалить
    num_to_remove = int(len(indices_one) * remove_fraction)
    
    # Случайным образом выбираем строки для удаления
    np.random.seed(42)  # для воспроизводимости
    indices_to_remove = np.random.choice(indices_one, num_to_remove, replace=False)
    
    # Удаляем выбранные строки из X и y
    mask = np.ones(len(y), dtype=bool)
    mask[indices_to_remove] = False
    
    X_balanced = X[mask]
    y_balanced = y[mask]
    
    return X_balanced, y_balanced

# Пример использования
# X_va - разряженная матрица признаков (csr_matrix)
# y_val - метки классов (numpy array)

# Удаляем 30% строк с меткой 1
X_balanced, y_balanced = adjust_class_balance(X_val, y_val, remove_fraction=0.3)

In [ ]:
X_balanced.shape

In [ ]:
%%time
from baseline import make_report
make_report()